# Case Studies in Risk Management 

###**GWP - S2 - G20**

##**1	Data Importing**

1.1 Choose a financial crisis of the last 100 years. Be sure that you can find data for the crisis you select. (Otherwise, select a different crisis)


* We decided to choose the Financial crisis of 2007–2008, as we have reliable data available.

1.2 Identify the key market factors


* Interest Rates
* Housing Prices
* Credit Risk
* Leverage Ratios of Investment banks
* Household Debt

##**2	Data Processing**

2.1 Find at least 5 securities, macroeconomic indicators, or other factors that were signficantly affected by the crisis.

After researching the 2007-2008 Financial crisis, we decided to import the following macroeconomic indicators:

* Federal Funds Effective Rate
* S&P/Case-Shiller U.S. National Home Price Index
* TED Spread
* Real gross domestic product per capita
* Household Debt Service Payments as a Percent of Disposable Personal Income
* Household Debt to GDP for United States
* Unemployment Rate

These indicators showed significant affectation from the crisis and in terms of the TED Spread, an indicator of perceived risk in the economy, it can reflect increases in credit risk.

For securities:
* SPY: SPDR S&P 500 ETF Trust
* FMCC: Federal Home Loan Mortgage Corporation
* ^TNX: Treasury Yield 10 Years
* IWM: Russel 2000 ETF
* USO: OIL ETF Trust
* AIG: American International Group, (Major insurance company)
* ^VIX: CBOE Volatility Index

With regards to the SPY and the IWM, we wanted to include the 500 biggest companies and the broader market. AIG almost took down the world economy. FMCC played a major role in the housing bubble as it was selling MBSs. ^TNX is the 10 Years Treasury Yield, which is not considered as a security. However it is simpler to import the Treasury Yield rather then incorporate a 10 year Treasury bond traded at that time. Since we know that Treasury Yields go up when bond prices go down and vice versa, this doesn't pose a technical inconsistency. USO tracks the oil prices and VIX meausres the volatility of the commodities index.

2.2 Be sure to collect data that starts 6 months prior to the crisis, and goes 6 months after the crisis.

2.3 Download them into Python

In [1]:
# IMPORT REQUIRED MODULES, PACKAGES AND METHODS

from sklearn.model_selection import train_test_split
from matplotlib.pyplot import figure
from sklearn.metrics import mean_squared_error
from scipy.stats import iqr
from sklearn import preprocessing
from fredapi import Fred

import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import yfinance as yf
import seaborn as sns
import pandas as pd
import numpy as np
import dateutil

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# INITIALISE THE FRED DATA-PULL

start = '2006-05-01'
end   = '2010-10-31'
fred  = Fred(api_key='09573ec1dc0c523e914163ab565c56e8')

* Macroeconomic indicators:

In [3]:
# DOWNLOAD 7 MACROECONOMIC VARIATES OF INTEREST AND RELEVANCE

DFF             = fred.get_series('DFF', observation_start = start, 
                                         observation_end   = end).resample('1m').nearest() # Federal Funds Effective Rate

CSUSHPINSA      = fred.get_series('CSUSHPINSA', observation_start = start, 
                                                observation_end   = end).resample('1m').nearest() # S&P/Case-Shiller U.S. National Home Price Index

TEDRATE         = fred.get_series('TEDRATE', observation_start = start, 
                                             observation_end   = end).resample('1m').nearest() # TED Spread

A939RX0Q048SBEA = fred.get_series('A939RX0Q048SBEA', observation_start = start, 
                                                     observation_end   = end).resample('1m').nearest() # Real gross domestic product per capita

TDSP            = fred.get_series('TDSP', observation_start = start, 
                                          observation_end   = end).resample('1m').nearest() # Household Debt Service Payments as a Percent of Disposable Personal Income

HDTGPDUSQ163N   = fred.get_series('HDTGPDUSQ163N', observation_start = start, 
                                                   observation_end   = end).resample('1m').nearest() # Household Debt to GDP for United States

UNRATE          = fred.get_series('UNRATE', observation_start = start, 
                                            observation_end   = end).resample('1m').nearest() # Unemployment Rate

Use the same securities as before, importing, aligning, and computing returns.

* Securities:

In [4]:
# IDENTIFY THE SECURITIES WE STUDY

securities      = 'SPY FMCC ^TNX IWM USO AIG ^VIX'

In [5]:
# INITIALISE THE SECURITIES DATA PULL

start = '2006-05-01'
end   = '2010-10-31'

data_securities = yf.download(securities, threads = True, start = start, end = end, interval = '1mo').dropna()

[*********************100%***********************]  7 of 7 completed


In [6]:
# DISPLAY THE SECURITIES DATA

data_securities['Adj Close'].head()

,AIG,FMCC,IWM,SPY,USO,^TNX,^VIX
Date,,,,,,,
2006-05-01,799.599487,55.461277,57.993015,93.135590,541.840027,5.113,16.440001
2006-06-01,778.511780,52.662350,57.880028,92.967598,558.159973,5.138,13.080000
2006-07-01,799.869629,53.880783,56.312050,93.796715,556.159973,4.988,14.950000
2006-08-01,841.398987,59.226025,58.001087,95.843590,518.400024,4.732,12.310000
2006-09-01,875.839844,61.768272,58.186947,98.000488,456.000000,4.633,11.980000


2.4	Be sure to align their frequencies (e.g. some may be daily, some may be quarterly)

In [7]:
# PREPARE THE MONTHLY MEV DATA AND DISPLAY IT

df_MACRO = pd.concat([DFF, CSUSHPINSA, TEDRATE, A939RX0Q048SBEA, TDSP, HDTGPDUSQ163N, UNRATE], axis = 1).dropna()
DF_MACRO = df_MACRO.apply(lambda x: x.resample('1m').ffill())

DF_MACRO.head()

,0,1,2,3,4,5,6
2006-05-31,5.05,184.547,0.50,51155.0,12.779304,96.907416,4.6
2006-06-30,5.05,184.607,0.61,51155.0,12.779304,96.907416,4.7
2006-07-31,5.31,184.404,0.50,51155.0,12.779304,96.907416,4.7
2006-08-31,5.31,184.198,0.48,51455.0,12.866873,98.906236,4.5
2006-09-30,5.34,184.055,0.60,51455.0,12.866873,98.906236,4.4


In [8]:
# STANDARDIZE THE MEV DATA BETWEEN [0,1] FOR USAGE IN FURTHER ANALYTICAL WORK

x              = DF_MACRO.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled       = min_max_scaler.fit_transform(x)
DF_MACRO       = pd.DataFrame(x_scaled)

DF_MACRO.columns = ['Fed Rate',
                    'Home Price Index', 
                    'TED Spread', 
                    'GDP', 
                    'Household Debt',
                    'Household Debt to GDP',
                    'Unemployment Rate']

macro_index    = data_securities["Adj Close"].dropna().index
DF_MACRO.index = macro_index

DF_MACRO.head()

,Fed Rate,Home Price Index,TED Spread,GDP,Household Debt,Household Debt to GDP,Unemployment Rate
Date,,,,,,,
2006-05-01,0.932836,0.998556,0.125413,0.657261,0.823015,0.414709,0.035714
2006-06-01,0.932836,1.000000,0.161716,0.657261,0.823015,0.414709,0.053571
2006-07-01,0.981343,0.995115,0.125413,0.657261,0.823015,0.414709,0.053571
2006-08-01,0.981343,0.990157,0.118812,0.770128,0.862410,0.703381,0.017857
2006-09-01,0.986940,0.986715,0.158416,0.770128,0.862410,0.703381,0.000000


In [9]:
# COMPUTE MONTHLY RETURNS FOR SECURITIES AND DISPLAY THE DATA

securities_return = data_securities["Adj Close"].dropna().pct_change().dropna()
securities_return.head()

,AIG,FMCC,IWM,SPY,USO,^TNX,^VIX
Date,,,,,,,
2006-06-01,-0.026373,-0.050466,-0.001948,-0.001804,0.030119,0.004890,-0.204380
2006-07-01,0.027434,0.023137,-0.027090,0.008918,-0.003583,-0.029194,0.142966
2006-08-01,0.051920,0.099205,0.029994,0.021822,-0.067894,-0.051323,-0.176589
2006-09-01,0.040933,0.042925,0.003204,0.022504,-0.120370,-0.020921,-0.026808
2006-10-01,0.013734,0.047785,0.062140,0.036053,-0.085965,-0.005828,-0.073456


##**3	Visualizing data**

3.1	Show 3 different multivariate displays

3.2	Show a scatterplot matrix of each pairwise returns

3.3	Show a cluster analysis of the returns to see how the groups form.

3.4	Run a principal components, and show the biplots for the first 3 pairs of components.

##**4	Data Summaries.   Extreme Values**

4.1	Compute means, standard deviations, skewness, and kurtosis for each distribution.

* Mean of the Macro data:

In [10]:
# COMPUTE AND PRINT MEANS - MEVs

DF_MACRO = DF_MACRO.drop("2006-05-01")
DF_MACRO.head()

DF_MACRO.mean()

Fed Rate                 0.420691
Home Price Index         0.468972
TED Spread               0.217573
GDP                      0.550790
Household Debt           0.658289
Household Debt to GDP    0.568183
Unemployment Rate        0.457547
dtype: float64

* Mean of the Securities data:

In [11]:
# COMPUTE AND PRINT MEANS - SECURITIES

securities_return.mean()

AIG     0.011435
FMCC   -0.026634
IWM     0.002857
SPY     0.001823
USO    -0.006377
^TNX   -0.008614
^VIX    0.026685
dtype: float64

* Standard deviations

In [12]:
# COMPUTE STANDARD DEVIATES - MEVs

DF_MACRO.std()

Fed Rate                 0.420480
Home Price Index         0.375080
TED Spread               0.217862
GDP                      0.345200
Household Debt           0.315608
Household Debt to GDP    0.329293
Unemployment Rate        0.408847
dtype: float64

In [13]:
# COMPUTE STANDARD DEVIATES - SECURITIES

securities_return.std()

AIG     0.446755
FMCC    0.450992
IWM     0.066458
SPY     0.053541
USO     0.106868
^TNX    0.089183
^VIX    0.229975
dtype: float64

* Compute the skewness of each series. 

In [14]:
# COMPUTE SKEW - MEVs

DF_MACRO.skew()

Fed Rate                 0.331417
Home Price Index         0.185660
TED Spread               1.562159
GDP                     -0.342669
Household Debt          -0.946337
Household Debt to GDP   -0.665184
Unemployment Rate        0.169449
dtype: float64

In [15]:
# COMPUTE SKEW - SECURITIES

securities_return.skew()

AIG     3.570838
FMCC    4.351244
IWM    -0.618003
SPY    -0.684825
USO    -0.390362
^TNX    0.133343
^VIX    1.541062
dtype: float64

* Compute the kurtosis.

In [16]:
# COMPUTE KURTOSIS - MEVs

DF_MACRO.kurtosis()

Fed Rate                -1.692805
Home Price Index        -1.665208
TED Spread               2.739038
GDP                     -1.402228
Household Debt          -0.495485
Household Debt to GDP   -1.116316
Unemployment Rate       -1.835998
dtype: float64

In [17]:
# COMPUTE KURTOSIS - SECURITY RETURNS

securities_return.kurtosis()

AIG     18.366365
FMCC    25.902946
IWM      0.978865
SPY      0.648894
USO      1.028343
^TNX     2.474280
^VIX     3.259207
dtype: float64

##**5 Data Modeling**

5.1	Pick 2 of the series.  Fit a copula to them.


In [29]:
from copulas.multivariate import GaussianMultivariate

In [30]:
copula = GaussianMultivariate()
copula.fit(securities_return[["AIG","SPY"]])

In [52]:
copula.to_dict()

{'covariance': [[1.0, 0.6384634219033284], [0.6384634219033284, 1.0]],
 'univariates': [{'df': 1.231202999913358,
   'loc': -0.014980536216563886,
   'scale': 0.09333347570457398,
   'type': 'copulas.univariate.student_t.StudentTUnivariate'},
  {'dataset': [-0.001803731945068332,
    0.008918341868239699,
    0.021822459397876637,
    0.0225043584388378,
    0.03605338058344332,
    0.019885213886444708,
    0.007756209502384426,
    0.02069586795457079,
    -0.0196175521025308,
    0.007592419182781862,
    0.048437934265165294,
    0.03391995325907371,
    -0.01884943111031734,
    -0.027135028699665442,
    0.012832722468217428,
    0.03380989255691946,
    0.018375146502701334,
    -0.03873284236730967,
    -0.016480057176880614,
    -0.05547455383033273,
    -0.0258428319952817,
    -0.013824501134879763,
    0.05284865065367894,
    0.015116721899961272,
    -0.08813679376636363,
    -0.004028955974134774,
    0.015453652105619664,
    -0.0993864780816911,
    -0.1603547967288832

5.2	Fit a non-Gaussian copula.


In [ ]:
from copulas.multivariate import VineCopula

In [54]:
regular = VineCopula('regular')
regular.fit(securities_return[["AIG","SPY"]])

In [55]:
regular.to_dict()

{'type': 'copulas.multivariate.vine.VineCopula',
 'vine_type': 'regular',
 'fitted': True,
 'n_sample': 53,
 'n_var': 2,
 'depth': 1,
 'truncated': 3,
 'trees': [{'tree_type': <TreeTypes.REGULAR: 2>,
   'type': 'copulas.multivariate.tree.RegularTree',
   'fitted': True,
   'level': 1,
   'n_nodes': 2,
   'tau_matrix': [[1.0, 0.5108853410740204], [0.5108853410740204, 1.0]],
   'previous_tree': [[0.5103266989301772, 0.42969036534225524],
    [0.5890103565895445, 0.5094388827171857],
    [0.6237185767353117, 0.6094792514758165],
    [0.6082691593403937, 0.6147254465998371],
    [0.5692055130480576, 0.7142822536262406],
    [0.616681793561755, 0.5945100606102082],
    [0.5804144089228834, 0.5005410802765914],
    [0.4831590177109618, 0.6007844553697076],
    [0.5203542713018827, 0.31490628374368435],
    [0.5551869755459728, 0.49929074546973523],
    [0.6069731172967617, 0.7928697093082291],
    [0.5994956702224788, 0.6993861321138402],
    [0.5054080307034892, 0.3193032808673126],
    [0.

5.3	Comment on the comparison of the fits.


##**6 Modeling Before and After the Crisis**

6.1	Using data pre-crisis, run any model from Machine Learning (LASSO, CART, PCA, NN, SVM, Bayes, or other ML techniques)

6.2	Using data post-crisis, run the same model.


6.3	Determine any mean-reverting relationships


6.4	How do the results compare to the results you achieved from the econometrics models you ran in the previous submission?
